In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
def get_top_stock():
    try:
        res=requests.get('https://cafef.vn/get-top-stock.chn')
        data=res.json()['stocks']
    except:
        return "Không thể tìm nạp dữ liệu"    
    return data 

def get_info_stock(stock_name):
    try:
        res=requests.get(f'https://e.cafef.vn/info.ashx?type=cp&symbol={stock_name}')
        data=res.json()
    except:
        return "Không thể tìm nạp dữ liệu hoặc mã cổ phiếu cần tìm bị sai"    
    return data 

In [4]:
def get_list_post(page_num):
    try:
        res=requests.get(f'https://cafef.vn/timelinelist/18831/{page_num}.chn')
        soup=BeautifulSoup(res.content,'html.parser')
        data=soup.find_all(class_='tlitem box-category-item')


    except Exception as e :
        return f"gặp lỗi : {e}"    
    return data 


In [5]:
def get_info__post(data):
    times=[]
    desc=[]
    links=[]
    titles=[]
    for i in range (len(data)):
        times.append(data[i].find(class_='time time-ago').text)
        desc.append(data[i].find(class_='sapo box-category-sapo').text)
        links.append('https://cafef.vn'+data[i].find('a',href=True)['href'])
        titles.append(data[i].find('a',href=True).text)

    return times,desc,links,titles

        

In [6]:
def get_content_post(link_post):
    content=''
    try:
        res=requests.get(link_post)
        soup=BeautifulSoup(res.content,'html.parser')
        c=soup.find(class_='detail-content afcbc-body')
        p=c.find_all('p')
        for i in range(len(p)):
            content=content+ p[i].text+ '\n'



    except Exception as e :
        return f"gặp lỗi : {e}"    
    return content 
    

In [7]:
column_names = ['Title', 'Desc', 'Content','Link','Time']  

df = pd.DataFrame(columns=column_names)

In [8]:
for i in range(1,15):
    content=[]
    times,desc,links,titles=get_info__post(get_list_post(i))
    for j in range(len(links)):
        content.append(get_content_post(links[j]))
       
    temp_df = pd.DataFrame({'Title':titles, 'Desc':desc, 'Content':content,'Link':links,'Time':times})
    df=pd.concat([df, temp_df])



In [9]:
df.head(5)

,Title,Desc,Content,Link,Time
0,"Khối ngoại mua ròng nhẹ trong phiên cuối tuần,...","Sau nhiều phiên mạnh tay xả ròng ngày 22/6, gi...","Xu hướng tích cực nối dài, VN-Index ghi nhận...",https://cafef.vn/khoi-ngoai-mua-rong-nhe-trong...,2023-06-23T15:54:00
1,Vinamilk bất ngờ giao dịch đột biến sau 17 năm...,"Sau 3 phiên liên tiếp hồi phục từ đáy một năm,...",Thị trường chứng khoán vừa khép lại phiên gi...,https://cafef.vn/vinamilk-bat-ngo-giao-dich-do...,2023-06-23T15:45:00
2,Dragon Capital tiếp tục gom thêm cổ phần Hoa S...,"Sau giao dịch, nhóm quỹ Dragon Capital tăng lư...",Nhóm quỹ Dragon Capital vừa có báo cáo về gi...,https://cafef.vn/dragon-capital-tiep-tuc-gom-t...,2023-06-23T15:08:00
3,Chủ tịch Louis Holdings Đỗ Thành Nhân kháng cá...,"Theo kháng cáo, ông Đỗ Thành Nhân đề nghị cấp ...","Ngày 24/6, được biết TAND TP Hà Nội vừa nhận...",https://cafef.vn/chu-tich-louis-holdings-do-th...,2023-06-23T10:24:00
4,"Đằng sau cuộc chiến giá rẻ: ""Chiêu trò"" truyền...","Theo chuyên gia truyền thông, Thế Giới Di Động...",Cạnh tranh qua thông điệp quảng cáo - chuyện k...,https://cafef.vn/dang-sau-cuoc-chien-gia-re-ch...,2023-06-23T08:01:00


In [ ]:
df.to_csv('dulieu.csv')